In [98]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import logging

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# Downloading Google Word2Vec

In [31]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

INFO - 11:51:55: Creating C:\Users\Aazim/gensim-data


[==================================================] 100.0% 1662.8/1662.8MB downloaded


INFO - 12:31:13: word2vec-google-news-300 downloaded


C:\Users\Aazim/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


In [34]:
google_w2v = KeyedVectors.load_word2vec_format(path,binary=True)

INFO - 12:42:11: loading projection weights from C:\Users\Aazim/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
INFO - 12:43:00: KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\Aazim/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-12-18T12:43:00.454393', 'gensim': '4.3.0', 'python': '3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


In [35]:
google_w2v.vector_size

300

# Importing Data
#### I have collected 15 paragraphs on different topics. I passes 10  parapraphs through a paraphrasing tool which generated a rephrased version of each paragraph to which I will later compare each of those 10 paragraphs. The other 5 paragraphs will compared with one of the other parapgraphs.


In [233]:
d11 = 'The advent of social media has transformed the way we communicate, enabling global connectivity. While it facilitates instant communication, concerns arise regarding its impact on personal relationships. Cyberbullying, misinformation, and the tendency for online interactions to replace face-to-face communication are noteworthy issues. Striking a balance between the benefits and pitfalls of social media is crucial. Recognizing its role in society, individuals and platforms must address challenges to foster healthy online and offline relationships.'
d21 = "Social media's introduction has revolutionized communication and made it possible for people to connect globally. Although it makes communication instantaneous, there are worries about how it will affect interpersonal connections. Notable problems include misinformation, cyberbullying, and the trend toward virtual interactions taking the place of in-person contacts. It's critical to strike a balance between social media's advantages and disadvantages. Acknowledging its place in society, people and platforms need to solve issues in order to promote positive interactions both online and offline."

d12 = 'The rapid evolution of Artificial Intelligence (AI) raises ethical questions about its deployment in various facets of society. Privacy concerns, biased algorithms, and the ethical responsibility of creators are central themes. Striking a balance between technological advancement and ethical considerations is imperative to ensure AI serves humanity without compromising fundamental values. A comprehensive ethical framework and ongoing dialogue are essential to navigate the evolving landscape of AI and its implications for the future.'
d22 = "The swift advancement of Artificial Intelligence (AI) gives rise to moral concerns regarding its application in diverse areas of society. The main issues include the ethical obligation of authors, privacy problems, and biased algorithms. To make sure AI benefits humanity without undermining core principles, it is crucial to strike a balance between ethical concerns and technological innovation. To successfully manage the rapidly changing field of artificial intelligence and its long-term consequences, a thorough ethical framework and constant communication are needed."

d13 = 'In the dynamic landscape of Science, Technology, Engineering, and Mathematics (STEM), the role of women is gaining prominence. Breaking gender stereotypes, women are making significant contributions to these fields. However, challenges such as gender bias and underrepresentation persist. Addressing these issues is crucial for fostering diversity and innovation. Recognizing and celebrating the achievements of women in STEM, coupled with proactive measures to eliminate barriers, will contribute to a more inclusive and equitable future.'
d23 = "Women are playing an increasingly important role in the ever-changing fields of science, technology, engineering, and mathematics (STEM). Women are defying gender norms and contributing significantly to various sectors. But issues like underrepresentation and gender bias continue. It is imperative to address these problems in order to promote variety and creativity. In addition to taking proactive steps to remove obstacles, recognizing and applauding the accomplishments of women in STEM can help create a more inclusive and fair future."

d14 = 'Climate change poses a formidable threat to global biodiversity, impacting ecosystems and species. Rising temperatures, extreme weather events, and habitat loss are key contributors to the decline of various flora and fauna. Conservation efforts are critical to mitigate these effects and preserve biodiversity. Through international cooperation, sustainable practices, and conservation initiatives, we can strive to safeguard the intricate web of life on Earth and ensure a sustainable future for generations to come.'
d24 = "Global biodiversity is severely threatened by climate change, which has an effect on ecosystems and species. Increasing temperatures, harsh weather, and habitat loss are the main causes of the decline in many types of plants and animals. To lessen these effects and protect biodiversity, conservation activities are essential. We may work to protect the complex web of life on Earth and guarantee a sustainable future for future generations by promoting international collaboration, sustainable practices, and conservation efforts."

d15 = 'The rise of e-commerce has reshaped the landscape of retail, challenging traditional brick-and-mortar stores. While e-commerce offers convenience and a global marketplace, traditional retailers face the challenge of adapting to changing consumer preferences. Striking a balance between online and offline retail, harnessing technology, and providing unique in-store experiences are essential for the coexistence and evolution of both retail models.'
d25 = "The emergence of e-commerce has altered the retail environment and put traditional brick and mortar establishments under pressure. While online shopping is convenient and provides access to a worldwide market, traditional businesses find it difficult to adjust to the shifting tastes of their customers. Maintaining a healthy balance between physical and virtual retail, utilizing technology, and offering distinctive in-store experiences are critical to the survival and development of both retail models."

d16 = 'Mental health awareness is a crucial aspect of societal well-being, yet stigma surrounding mental illnesses persists. Breaking down these barriers requires collective efforts to educate, advocate, and create an empathetic environment. By fostering understanding and support, we can dismantle stereotypes and empower individuals to seek help without fear of judgment, ultimately contributing to a society that prioritizes mental health.'
d26 = "Despite the fact that stigma associated with mental diseases still exists, mental health awareness is vital to the wellbeing of society. It will take a team effort to lobby, educate, and foster empathy in order to remove these obstacles. In the end, we can contribute to a society that prioritizes mental health by eradicating stereotypes and empowering people to seek treatment without fear of judgment by encouraging understanding and support."

d17 = 'The advent of remote work in the digital age has revolutionized the traditional work landscape. While offering flexibility and work-life balance, challenges such as potential isolation and blurred boundaries between professional and personal life emerge. Striking a balance through effective communication, technological solutions, and policies that prioritize employee well-being is essential for harnessing the benefits of remote work while addressing its associated challenges.'
d27 = "The traditional work environment has undergone a metamorphosis with the rise of remote work in the digital age. Although it provides flexibility and a work-life balance, there are drawbacks, including the possibility of isolation and a blurring of the lines between personal and professional life. Harnessing the benefits of remote work while tackling its related issues requires striking a balance through efficient communication, technology solutions, and policies that promote employee well-being."

d18 = 'Pop culture serves as a powerful force shaping societal norms, attitudes, and values. From fashion trends to entertainment choices, its impact is pervasive. Examining this dynamic relationship allows us to understand how pop culture both reflects and influences society. Recognizing the reciprocal nature of this influence is essential for cultivating a cultural landscape that fosters inclusivity, creativity, and positive social change.'
d28 = "Pop culture exerts a strong influence on the attitudes, values, and conventions of society. Its influence may be seen in anything from entertainment choices to fashion trends. By looking at this dynamic interaction, we can comprehend how pop culture influences and reflects society. Understanding that this influence is reciprocal is crucial to creating a cultural environment that supports diversity, innovation, and constructive social change."

d19 = 'Cultural diversity enriches the educational experience by exposing individuals to different perspectives, traditions, and ways of thinking. Embracing diversity in classrooms fosters understanding, tolerance, and a global mindset. Education plays a pivotal role in promoting inclusivity, and curriculum design, teaching methods, and institutional policies should reflect a commitment to embracing and celebrating cultural diversity.'
d29 = "Cultural diversity exposes people to a variety of viewpoints, customs, and ways of thinking, which enhances the educational experience. Accepting variety in the classroom promotes tolerance, global awareness, and understanding. Education is essential for fostering tolerance, and institutional policies, instructional strategies, and curriculum design should all demonstrate a dedication to accepting and appreciating cultural variety."

d110 = 'The future of space exploration holds profound implications for humanity. As technological advancements enable us to venture farther into the cosmos, questions of ethical responsibility, environmental impact, and the potential for human colonization arise. Navigating these challenges requires international collaboration, ethical considerations, and a commitment to sustainable practices to ensure the responsible exploration and potential habitation of celestial bodies.'
d210 = "There are significant ramifications for humanity in the future of space travel. The ability to travel further into space thanks to technical improvements raises concerns about environmental damage, ethical responsibility, and the possibility of human colonization. In order to assure the responsible exploration and possible habitation of celestial planets, navigating these issues will require international collaboration, ethical considerations, and a commitment to sustainable practices."

d111 = 'In the face of climate change, the adoption of renewable energy sources is crucial for reducing carbon emissions and mitigating environmental impact. Exploring the benefits of solar, wind, and other sustainable energy options, this essay emphasizes the importance of transitioning to renewable energy to achieve a more sustainable and environmentally conscious future.'
d211 = 'Language evolves alongside technological advancements, particularly in the digital era. This essay explores how digital platforms and communication technologies influence language use, introducing new forms of expression and communication. While acknowledging the benefits, it also reflects on the challenges of maintaining linguistic richness and cultural nuances in the fast-paced, digital communication landscape.'

d112 = 'The influence of video games on cognitive skills and behavior is a subject of ongoing debate. While some argue that video games can enhance cognitive abilities, others raise concerns about potential negative effects on behavior. This essay explores the nuanced relationship between video games and the human mind, acknowledging both the potential benefits and challenges associated with gaming.'
d212 = 'As the global population ages, societies face both challenges and opportunities. This essay delves into the implications of an aging population on healthcare systems, social structures, and economic dynamics. Exploring innovative solutions and highlighting the potential for intergenerational collaboration, it emphasizes the need to adapt and create inclusive environments that cater to the diverse needs of all age groups.'

d113 = 'Education plays a pivotal role in shaping attitudes and behaviors toward the environment. This essay discusses how educational institutions can contribute to fostering environmental sustainability by integrating ecological awareness into curricula, promoting sustainable practices, and instilling a sense of responsibility for the well-being of the planet among students.'
d213 = 'The influence of video games on cognitive skills and behavior is a subject of ongoing debate. While some argue that video games can enhance cognitive abilities, others raise concerns about potential negative effects on behavior. This essay explores the nuanced relationship between video games and the human mind, acknowledging both the potential benefits and challenges associated with gaming.'

d114 = 'As the global population ages, societies face both challenges and opportunities. This essay delves into the implications of an aging population on healthcare systems, social structures, and economic dynamics. Exploring innovative solutions and highlighting the potential for intergenerational collaboration, it emphasizes the need to adapt and create inclusive environments that cater to the diverse needs of all age groups.'
d214 = 'Education plays a pivotal role in shaping attitudes and behaviors toward the environment. This essay discusses how educational institutions can contribute to fostering environmental sustainability by integrating ecological awareness into curricula, promoting sustainable practices, and instilling a sense of responsibility for the well-being of the planet among students.'

d115 = 'Language evolves alongside technological advancements, particularly in the digital era. This essay explores how digital platforms and communication technologies influence language use, introducing new forms of expression and communication. While acknowledging the benefits, it also reflects on the challenges of maintaining linguistic richness and cultural nuances in the fast-paced, digital communication landscape.'
d215 = 'In the face of climate change, the adoption of renewable energy sources is crucial for reducing carbon emissions and mitigating environmental impact. Exploring the benefits of solar, wind, and other sustainable energy options, this essay emphasizes the importance of transitioning to renewable energy to achieve a more sustainable and environmentally conscious future.'



In [243]:
docs1 = [d11, d12, d13, d14, d15, d16, d17, d18, d19, d110, d111, d112, d113, d114, d115]
docs2 = [d21, d22, d23, d24, d25, d26, d27, d28, d29, d210, d211, d212, d213, d214, d215]
labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [235]:
df = pd.DataFrame(columns=['doc1','doc2','label'])
df['doc1'] = pd.Series(docs1)
df['doc2'] = pd.Series(docs2)
df['label'] = pd.Series(labels)
df.head()

,doc1,doc2,label
0,The advent of social media has transformed the...,Social media's introduction has revolutionized...,1
1,The rapid evolution of Artificial Intelligence...,The swift advancement of Artificial Intelligen...,1
2,"In the dynamic landscape of Science, Technolog...",Women are playing an increasingly important ro...,1
3,Climate change poses a formidable threat to gl...,Global biodiversity is severely threatened by ...,1
4,The rise of e-commerce has reshaped the landsc...,The emergence of e-commerce has altered the re...,1


# Exploring Data

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doc1    15 non-null     object
 1   doc2    15 non-null     object
 2   label   15 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 492.0+ bytes


In [236]:
df['label'].value_counts()

1    10
0     5
Name: label, dtype: int64

In [224]:
df.isna().sum()

doc1     0
doc2     0
label    0
dtype: int64

In [201]:
df.dropna(inplace=True)

# Pre-processing Text
#### Pre-processing the text by removing punctuations, special characters, stopwords and then lemmatizing.

In [237]:
english_stops = set(stopwords.words('english'))
punctuations = set(punctuation)
stop_words_punc = english_stops.union(punctuations)

lemmatizer = WordNetLemmatizer()

def remove_stopwords_punc_lemmatize(text):
    tokens = word_tokenize(text)
    filtered_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens if token not in stop_words_punc])
    return filtered_text
    
def remove_special_characters(text):
    pattern=r'[^a-z\s]'
    text=re.sub(pattern,'',text)
    return text

def text_preprocessing(text):
    text = text.lower()
    text = remove_special_characters(text)
    text = remove_stopwords_punc_lemmatize(text)
    text = text.strip()
    return text

df['length'] = df['doc1'].str.len()+df['doc2'].str.len()
df['proc_doc1'] = df['doc1'].apply(text_preprocessing)
df['proc_doc2'] = df['doc2'].apply(text_preprocessing)
df['proc_length'] = df['proc_doc1'].str.len()+df['proc_doc2'].str.len()
df.head()

,doc1,doc2,label,length,proc_doc1,proc_doc2,proc_length
0,The advent of social media has transformed the...,Social media's introduction has revolutionized...,1,1152,advent social medium transformed way communica...,social medias introduction revolutionized comm...,898
1,The rapid evolution of Artificial Intelligence...,The swift advancement of Artificial Intelligen...,1,1112,rapid evolution artificial intelligence ai rai...,swift advancement artificial intelligence ai g...,887
2,"In the dynamic landscape of Science, Technolog...",Women are playing an increasingly important ro...,1,1080,dynamic landscape science technology engineeri...,woman playing increasingly important role ever...,827
3,Climate change poses a formidable threat to gl...,Global biodiversity is severely threatened by ...,1,1049,climate change pose formidable threat global b...,global biodiversity severely threatened climat...,807
4,The rise of e-commerce has reshaped the landsc...,The emergence of e-commerce has altered the re...,1,954,rise ecommerce reshaped landscape retail chall...,emergence ecommerce altered retail environment...,729


In [238]:
df.describe()

,label,length,proc_length
count,15.000000,15.000000,15.000000
mean,0.666667,924.266667,716.666667
std,0.487950,128.759725,102.609987
min,0.000000,765.000000,582.000000
25%,0.000000,806.500000,623.000000
50%,1.000000,883.000000,701.000000
75%,1.000000,1014.500000,782.000000
max,1.000000,1152.000000,898.000000


# Generating Doc Vectors
#### Genereated Doc Vectors by averaging word vectors of each word in a document. Used Google Word2Vec to extract word vectors.

In [239]:
def get_doc_vec_google(text):
    tokens = word_tokenize(text)
    doc_vec = []
    for token in tokens:
        try:
            word_vec = google_w2v[token]
            doc_vec.append(word_vec)
        except:
            pass
    return np.mean(doc_vec,axis=0)

df['doc1_vec_google'] = df['proc_doc1'].apply(get_doc_vec_google)
df['doc2_vec_google'] = df['proc_doc2'].apply(get_doc_vec_google)
df.head()

,doc1,doc2,label,length,proc_doc1,proc_doc2,proc_length,doc1_vec_google,doc2_vec_google
0,The advent of social media has transformed the...,Social media's introduction has revolutionized...,1,1152,advent social medium transformed way communica...,social medias introduction revolutionized comm...,898,"[0.038906667, 0.0013414748, -0.07879119, 0.053...","[0.034070183, -0.019756243, -0.05070451, 0.022..."
1,The rapid evolution of Artificial Intelligence...,The swift advancement of Artificial Intelligen...,1,1112,rapid evolution artificial intelligence ai rai...,swift advancement artificial intelligence ai g...,887,"[-0.0022934277, 0.042434692, -0.028642973, 0.0...","[-0.027488016, 0.016946273, 0.0044371174, 0.05..."
2,"In the dynamic landscape of Science, Technolog...",Women are playing an increasingly important ro...,1,1080,dynamic landscape science technology engineeri...,woman playing increasingly important role ever...,827,"[-0.020990668, 0.04442003, -0.013234795, 0.012...","[-0.0014760543, 0.044021294, -0.019958185, 0.0..."
3,Climate change poses a formidable threat to gl...,Global biodiversity is severely threatened by ...,1,1049,climate change pose formidable threat global b...,global biodiversity severely threatened climat...,807,"[-0.026077392, 0.0920803, -0.040027536, 0.0221...","[-0.0254637, 0.0755628, -0.012654066, 0.031477..."
4,The rise of e-commerce has reshaped the landsc...,The emergence of e-commerce has altered the re...,1,954,rise ecommerce reshaped landscape retail chall...,emergence ecommerce altered retail environment...,729,"[0.051001266, 0.0337079, -0.11178176, 0.049172...","[0.019102229, 0.04611188, -0.12749517, 0.05138..."


In [228]:
df.isna().sum()

doc1               0
doc2               0
label              0
length             0
proc_doc1          0
proc_doc2          0
proc_length        0
doc1_vec_google    0
doc2_vec_google    0
dtype: int64

In [206]:
df.dropna(inplace=True)

# Cosine Similarity
#### Calculated cosine similarity to measure simiarities bewteen documents.

In [240]:
dists = []
for row in  df.itertuples():
    dist = cosine_similarity([row.doc1_vec_google, row.doc2_vec_google])
    dists.append(dist[0, 1])


df['dist'] = pd.Series(dists)

In [241]:
df['pred'] = np.where(df['dist']<0.9, 0, 1)
df.head(15)

,doc1,doc2,label,length,proc_doc1,proc_doc2,proc_length,doc1_vec_google,doc2_vec_google,dist,pred
0,The advent of social media has transformed the...,Social media's introduction has revolutionized...,1,1152,advent social medium transformed way communica...,social medias introduction revolutionized comm...,898,"[0.038906667, 0.0013414748, -0.07879119, 0.053...","[0.034070183, -0.019756243, -0.05070451, 0.022...",0.926847,1
1,The rapid evolution of Artificial Intelligence...,The swift advancement of Artificial Intelligen...,1,1112,rapid evolution artificial intelligence ai rai...,swift advancement artificial intelligence ai g...,887,"[-0.0022934277, 0.042434692, -0.028642973, 0.0...","[-0.027488016, 0.016946273, 0.0044371174, 0.05...",0.936110,1
2,"In the dynamic landscape of Science, Technolog...",Women are playing an increasingly important ro...,1,1080,dynamic landscape science technology engineeri...,woman playing increasingly important role ever...,827,"[-0.020990668, 0.04442003, -0.013234795, 0.012...","[-0.0014760543, 0.044021294, -0.019958185, 0.0...",0.932933,1
3,Climate change poses a formidable threat to gl...,Global biodiversity is severely threatened by ...,1,1049,climate change pose formidable threat global b...,global biodiversity severely threatened climat...,807,"[-0.026077392, 0.0920803, -0.040027536, 0.0221...","[-0.0254637, 0.0755628, -0.012654066, 0.031477...",0.956953,1
4,The rise of e-commerce has reshaped the landsc...,The emergence of e-commerce has altered the re...,1,954,rise ecommerce reshaped landscape retail chall...,emergence ecommerce altered retail environment...,729,"[0.051001266, 0.0337079, -0.11178176, 0.049172...","[0.019102229, 0.04611188, -0.12749517, 0.05138...",0.913566,1
5,Mental health awareness is a crucial aspect of...,Despite the fact that stigma associated with m...,1,880,mental health awareness crucial aspect societa...,despite fact stigma associated mental disease ...,692,"[0.007729135, 0.054304447, -0.047068898, 0.045...","[-0.021805536, 0.06424704, -0.03673517, 0.0619...",0.957751,1
6,The advent of remote work in the digital age h...,The traditional work environment has undergone...,1,980,advent remote work digital age revolutionized ...,traditional work environment undergone metamor...,733,"[0.008107185, 0.02638197, -0.052975465, 0.0698...","[0.015235538, 0.049222946, -0.05874779, 0.0586...",0.945280,1
7,Pop culture serves as a powerful force shaping...,Pop culture exerts a strong influence on the a...,1,883,pop culture serf powerful force shaping societ...,pop culture exerts strong influence attitude v...,683,"[0.004456869, 0.046908494, -0.037891667, 0.051...","[0.012232072, 0.042136166, -0.030242138, 0.037...",0.953026,1
8,Cultural diversity enriches the educational ex...,Cultural diversity exposes people to a variety...,1,866,cultural diversity enriches educational experi...,cultural diversity expose people variety viewp...,701,"[-0.037422583, 0.06263331, 0.059970893, 0.0995...","[-0.07119027, 0.03426997, -0.024632521, 0.1010...",0.928257,1
9,The future of space exploration holds profound...,There are significant ramifications for humani...,1,962,future space exploration hold profound implica...,significant ramification humanity future space...,757,"[-0.07704697, 0.026492309, 0.028681945, 0.0454...","[-0.047617596, 0.05901004, 0.009059808, 0.0555...",0.947467,1


# Evaluation

In [242]:
print(classification_report(df['label'], df['pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        10

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

